## Deduplicating the febrl3 dataset

See A.2 [here](https://arxiv.org/pdf/2008.04443.pdf) and [here](https://recordlinkage.readthedocs.io/en/latest/ref-datasets.html) for the source of this data


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/master/docs/demos/examples/duckdb/febrl3.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
!pip install splink

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 60.3 MB/s eta 0:00:00


In [2]:
from splink.datasets import splink_datasets

df = splink_datasets.febrl3

downloading: https://raw.githubusercontent.com/moj-analytical-services/splink_datasets/master/data/febrl/dataset3.csv



In [3]:
df = df.rename(columns=lambda x: x.strip())

df["cluster"] = df["rec_id"].apply(lambda x: "-".join(x.split("-")[:2]))

df["date_of_birth"] = df["date_of_birth"].astype(str).str.strip()
df["soc_sec_id"] = df["soc_sec_id"].astype(str).str.strip()

df.head(2)

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id,cluster
0,rec-1496-org,mitchell,green,7,wallaby place,delmar,cleveland,2119,sa,19560409,1804974,rec-1496
1,rec-552-dup-3,harley,mccarthy,177,pridhamstreet,milton,marsden,3165,nsw,19080419,6089216,rec-552


In [4]:
df["date_of_birth"] = df["date_of_birth"].astype(str).str.strip()
df["soc_sec_id"] = df["soc_sec_id"].astype(str).str.strip()

In [5]:
df["date_of_birth"] = df["date_of_birth"].astype(str).str.strip()
df["soc_sec_id"] = df["soc_sec_id"].astype(str).str.strip()

In [6]:
from splink import DuckDBAPI, Linker, SettingsCreator

# TODO:  Allow missingness to be analysed without a linker
settings = SettingsCreator(
    unique_id_column_name="rec_id",
    link_type="dedupe_only",
)

linker = Linker(df, settings, db_api=DuckDBAPI())

It's usually a good idea to perform exploratory analysis on your data so you understand what's in each column and how often it's missing:


In [7]:
from splink.exploratory import completeness_chart

completeness_chart(df, db_api=DuckDBAPI())

alt.LayerChart(...)

In [8]:
from splink.exploratory import profile_columns

profile_columns(df, db_api=DuckDBAPI(), column_expressions=["given_name", "surname"])

alt.VConcatChart(...)

In [9]:
from splink import DuckDBAPI, block_on
from splink.blocking_analysis import (
    cumulative_comparisons_to_be_scored_from_blocking_rules_chart,
)

blocking_rules = [
    block_on("soc_sec_id"),
    block_on("given_name"),
    block_on("surname"),
    block_on("date_of_birth"),
    block_on("postcode"),
]

db_api = DuckDBAPI()
cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=df,
    blocking_rules=blocking_rules,
    db_api=db_api,
    link_type="dedupe_only",
    unique_id_column_name="rec_id",
)

alt.Chart(...)

In [52]:
import splink.comparison_library as cl
import splink.comparison_level_library as cll

from splink import Linker

settings = SettingsCreator(
    unique_id_column_name="rec_id",
    link_type="dedupe_only",
    blocking_rules_to_generate_predictions=blocking_rules,
    probability_two_random_records_match=0.03589,
    comparisons=[
      cl.CustomComparison(comparison_levels=[
        cll.NullLevel("given_name"),
        cll.LevenshteinLevel("given_name", 2).configure(
            m_probability=0.5,
            fix_m_probability=True,
            u_probability=0.25,
            fix_u_probability=True,
        ),
        cll.ElseLevel().configure(
            m_probability=0,
            fix_m_probability=True,
            u_probability=0,
            fix_u_probability=True,
        )]),
      cl.CustomComparison(comparison_levels=[
        cll.NullLevel("surname"),
        cll.LevenshteinLevel("surname", 2).configure(
            m_probability=0.5,
            fix_m_probability=True,
            u_probability=0.25,
            fix_u_probability=True,
        ),cll.ElseLevel().configure(
            m_probability=0,
            fix_m_probability=True,
            u_probability=0,
            fix_u_probability=True,
        )]),
      cl.CustomComparison(comparison_levels=[
        cll.NullLevel("date_of_birth"),
        cll.AbsoluteDateDifferenceLevel(
            "date_of_birth",
            input_is_string=True,
            threshold=0,
            metric='day',
            datetime_format="%Y%m%d",
        ).configure(
            m_probability=0.5,
            fix_m_probability=True,
            u_probability=0.25,
            fix_u_probability=True,
        ),cll.ElseLevel().configure(
            m_probability=0,
            fix_m_probability=True,
            u_probability=0,
            fix_u_probability=True,
        )]),
      cl.CustomComparison(comparison_levels=[
          cll.NullLevel("soc_sec_id"),
          cll.DamerauLevenshteinLevel("soc_sec_id", 2).configure(
            m_probability=0.5,
            fix_m_probability=True,
            u_probability=0.25,
            fix_u_probability=True,
        ),cll.ElseLevel().configure(
            m_probability=0,
            fix_m_probability=True,
            u_probability=0,
            fix_u_probability=True,
        )]),
      cl.CustomComparison(comparison_levels=[
          cll.NullLevel("street_number"),
          cll.ExactMatchLevel("street_number", term_frequency_adjustments=False).configure(
            m_probability=0.5,
            fix_m_probability=True,
            u_probability=0.25,
            fix_u_probability=True,
        ),cll.ElseLevel().configure(
            m_probability=0,
            fix_m_probability=True,
            u_probability=0,
            fix_u_probability=True,
        )]),
        cl.CustomComparison(comparison_levels=[
            cll.NullLevel("postcode"),
            cll.ExactMatchLevel("postcode",term_frequency_adjustments=False).configure(
            m_probability=0.5,
            fix_m_probability=True,
            u_probability=0.25,
            fix_u_probability=True,
        ),cll.ElseLevel().configure(
            m_probability=0,
            fix_m_probability=True,
            u_probability=0,
            fix_u_probability=True,
        )]),
    ],
    retain_intermediate_calculation_columns=True,
)

linker = Linker(df, settings, db_api=DuckDBAPI())

ZeroDivisionError: float division by zero

In [50]:
from splink import block_on

deterministic_rules = [
    block_on("soc_sec_id"),
    block_on("given_name", "surname", "date_of_birth"),
    "l.given_name = r.surname and l.surname = r.given_name and l.date_of_birth = r.date_of_birth",
]

#linker.training.estimate_probability_two_random_records_match(
#    deterministic_rules, recall=0.9
#)

In [12]:
#linker.training.estimate_u_using_random_sampling(max_pairs=1e6)

INFO:splink.internals.estimate_u:----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

INFO:splink.internals.m_u_records_to_parameters:u probability not trained for date_of_birth - Abs date difference <= 1 month (comparison vector value: 3). This usually means the comparison level was never observed in the training data.
INFO:splink.internals.m_u_records_to_parameters:u probability not trained for date_of_birth - Abs date difference <= 1 year (comparison vector value: 2). This usually means the comparison level was never observed in the training data.
INFO:splink.internals.m_u_records_to_parameters:u probability not trained for date_of_birth - Abs date difference <= 10 year (comparison vector value: 1). This usually means the comparison level was never observed in the training data.
INFO:splink.internals.estimate_u:
Estimated u probabilities using random sampling
INFO:splink.internals.settings:
Your model is not yet fully trained. Missing estimates for:
    - given_name (no m values are trained).
    - surname (no m values are trained).
    - date_of_birth (some u values

In [25]:
em_blocking_rule_1 = block_on("date_of_birth")
session_dob = linker.training.estimate_parameters_using_expectation_maximisation(
    em_blocking_rule_1
)

INFO:splink.internals.em_training_session:
----- Starting EM training session -----

INFO:splink.internals.em_training_session:Estimating the m probabilities of the model by blocking on:
l."date_of_birth" = r."date_of_birth"

Parameter estimates will be made for the following comparison(s):
    - given_name
    - surname
    - soc_sec_id
    - street_number
    - postcode

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - date_of_birth


SplinkException: Error executing the following sql for table `__splink__df_comparison_vectors`(__splink__df_comparison_vectors_997b17e30):
CREATE TABLE __splink__df_comparison_vectors_997b17e30 AS
WITH __splink__blocked_id_pairs AS (
  SELECT
    *
  FROM __splink__blocked_id_pairs_d5b406a95
), __splink__df_concat_with_tf AS (
  SELECT
    *
  FROM __splink__df_concat_with_tf_6af34fc78
), blocked_with_cols AS (
  SELECT
    "l"."rec_id" AS "rec_id_l",
    "r"."rec_id" AS "rec_id_r",
    "l"."given_name" AS "given_name_l",
    "r"."given_name" AS "given_name_r",
    "l"."surname" AS "surname_l",
    "r"."surname" AS "surname_r",
    "l"."soc_sec_id" AS "soc_sec_id_l",
    "r"."soc_sec_id" AS "soc_sec_id_r",
    "l"."street_number" AS "street_number_l",
    "r"."street_number" AS "street_number_r",
    "l"."tf_street_number" AS "tf_street_number_l",
    "r"."tf_street_number" AS "tf_street_number_r",
    "l"."postcode" AS "postcode_l",
    "r"."postcode" AS "postcode_r",
    "l"."tf_post

In [ ]:
em_blocking_rule_2 = block_on("postcode")
session_postcode = linker.training.estimate_parameters_using_expectation_maximisation(
    em_blocking_rule_2
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."postcode" = r."postcode"

Parameter estimates will be made for the following comparison(s):
    - given_name
    - surname
    - date_of_birth
    - soc_sec_id
    - street_number

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - postcode

Level Abs difference of 'transformed date_of_birth <= 1 month' on comparison date_of_birth not observed in dataset, unable to train m value

Level Abs difference of 'transformed date_of_birth <= 1 year' on comparison date_of_birth not observed in dataset, unable to train m value

Level Abs difference of 'transformed date_of_birth <= 10 year' on comparison date_of_birth not observed in dataset, unable to train m value

Iteration 1: Largest change in params was 0.0681 in probability_two_random_records_match
Iteration 2: Largest change in params was -0.00185 in the m_probability of 

In [51]:
linker.visualisations.match_weights_chart()

alt.VConcatChart(...)

In [48]:
results = linker.inference.predict(threshold_match_probability=0.2)

INFO:splink.internals.linker_components.inference:Blocking time: 0.09 seconds
INFO:splink.internals.linker_components.inference:Predict time: 0.64 seconds
 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'given_name':
    m values not fully trained
Comparison: 'given_name':
    u values not fully trained
Comparison: 'surname':
    m values not fully trained
Comparison: 'surname':
    u values not fully trained
Comparison: 'date_of_birth':
    m values not fully trained
Comparison: 'date_of_birth':
    u values not fully trained
Comparison: 'soc_sec_id':
    m values not fully trained
Comparison: 'soc_sec_id':
    u values not fully trained
Comparison: 'street_number':
    m values not fully trained
Comparison: 'street_number':
    u values not fully trained
Comparison: 'postcod

In [42]:
linker.evaluation.accuracy_analysis_from_labels_column(
    "cluster", match_weight_round_to_nearest=0.1, output_type="accuracy"
)

INFO:splink.internals.linker_components.inference:Blocking time: 0.20 seconds
INFO:splink.internals.linker_components.inference:Predict time: 1.70 seconds


alt.LayerChart(...)

In [43]:
pred_errors_df = linker.evaluation.prediction_errors_from_labels_column(
    "cluster"
).as_pandas_dataframe()
len(pred_errors_df)
pred_errors_df.head()

INFO:splink.internals.linker_components.inference:Blocking time: 0.16 seconds
INFO:splink.internals.linker_components.inference:Predict time: 0.21 seconds


,clerical_match_score,found_by_blocking_rules,match_weight,match_probability,rec_id_l,rec_id_r,given_name_l,given_name_r,gamma_given_name,bf_given_name,...,postcode_l,postcode_r,gamma_postcode,tf_postcode_l,tf_postcode_r,bf_postcode,bf_tf_adj_postcode,cluster_l,cluster_r,match_key
0,1.0,True,-1.321784,0.285735,rec-1695-dup-3,rec-1695-org,lydia,lydia,1,2.0,...,4670,4670,1,0.0040,0.0040,2.0,62.500000,rec-1695,rec-1695,0
1,1.0,True,-4.374895,0.045981,rec-1929-dup-1,rec-1929-org,cheree,cheree,1,2.0,...,6111,4216,0,0.0002,0.0018,2.0,1.000000,rec-1929,rec-1929,0
2,1.0,True,-7.287568,0.006360,rec-427-dup-3,rec-427-dup-4,emmet,emmet,1,2.0,...,4365,4350,0,0.0002,0.0032,2.0,1.000000,rec-427,rec-427,0
3,1.0,True,-0.459287,0.421077,rec-1498-dup-0,rec-1498-dup-2,isabelle,kaminskas,0,2.0,...,2765,2765,1,0.0022,0.0022,2.0,113.636364,rec-1498,rec-1498,0
4,1.0,True,-1.857837,0.216233,rec-1917-dup-3,rec-1917-dup-4,angus,angus,1,2.0,...,6210,6210,1,0.0058,0.0058,2.0,43.103448,rec-1917,rec-1917,0


The following chart seems to suggest that, where the model is making errors, it's because the data is corrupted beyond recognition and no reasonable linkage model could find these matches

In [44]:
records = linker.evaluation.prediction_errors_from_labels_column(
    "cluster"
).as_record_dict(limit=10)
linker.visualisations.waterfall_chart(records)

INFO:splink.internals.linker_components.inference:Blocking time: 0.24 seconds
INFO:splink.internals.linker_components.inference:Predict time: 0.21 seconds


alt.LayerChart(...)